In [ ]:
import asyncio
%pip install ballsort
from ballsort.ballsort_display_utils import open_bs_window
open_bs_window()

In [ ]:
from control_factory import get_control_sim
from ch12_scenario import Ch12Scenario

In [ ]:
from state_update_model import StatePosition
from ball_control import BallControl

In [ ]:
async def noop():
    pass

async def go_to_pos(bc: BallControl, dest: StatePosition, open_claw: bool, claw_index: int = 0):
    rel_x = dest.x - bc.get_position(claw_index=claw_index).x
    rel_y = dest.y - bc.get_position(claw_index=claw_index).y
    await asyncio.gather(
        bc.move_horizontally(rel_x, claw_index=claw_index),
        bc.move_vertically(rel_y, claw_index=claw_index),
        bc.open_claw(claw_index=claw_index) if open_claw else noop(),
    )

async def move_ball(
    bc: BallControl, src: StatePosition, dest: StatePosition, claw_index: int = 0
):
    await go_to_pos(bc=bc, dest=src, open_claw=True, claw_index=claw_index)
    await bc.close_claw(claw_index=claw_index)
    await go_to_pos(bc=bc, dest=dest, open_claw=False, claw_index=claw_index)
    await bc.open_claw(claw_index=claw_index)

def get_column_top_occupied_y(bc: BallControl, x: int) -> int:
    return min(
        [ball.pos.y for ball in bc.get_state().balls if ball.pos.x == x],
        default=bc.get_state().max_y,
    )

def get_column_top_occupied_pos(bc: BallControl, x: int) -> StatePosition:
    return StatePosition(x=x, y=get_column_top_occupied_y(bc=bc, x=x))

def get_column_top_vacant_y(bc: BallControl, x: int) -> int:
    return (
        min(
            [ball.pos.y for ball in bc.get_state().balls if ball.pos.x == x],
            default=bc.get_state().max_y + 1,
        )
        - 1
    )

def get_column_top_vacant_pos(bc: BallControl, x: int) -> StatePosition:
    return StatePosition(x=x, y=get_column_top_vacant_y(bc=bc, x=x))

async def move_ball_by_column(
    bc: BallControl, src_x: int, dest_x: int, claw_index: int = 0
):
    await move_ball(
        bc=bc,
        src=get_column_top_occupied_pos(bc=bc, x=src_x),
        dest=get_column_top_vacant_pos(bc=bc, x=dest_x),
        claw_index=claw_index,
    )


In [ ]:
async def reveal_color_values(
    bc: BallControl,
    color_to_x: dict[str, int],
    color_to_event: dict[str, asyncio.Event],
    goal_nof_colors: int
):
    """reveal all color values with claw 1"""

    nof_balls = 6  # to reveal
    right_src_x = bc.get_state().max_x - 1
    reveal_x = bc.get_state().max_x
    scrap_x = bc.get_state().max_x - 2

    for _ in range(nof_balls):
        # move to reveal spot (can be done conditionally if the color is already known)
        await move_ball_by_column(
            bc=bc, src_x=right_src_x, dest_x=reveal_x, claw_index=1
        )

        # add revealed color to dict
        revealed_ball = next(
            ball for ball in bc.get_state().balls if ball.pos.x == reveal_x
        )
        assert revealed_ball
        assert revealed_ball.value
        color_to_x[revealed_ball.color] = revealed_ball.value
        ev = color_to_event.get(revealed_ball.color)
        if ev:
            ev.set()
        if len(color_to_x) >= goal_nof_colors:
            break
        # move to scrap heap column
        await move_ball_by_column(bc=bc, src_x=reveal_x, dest_x=scrap_x, claw_index=1)


async def sort_into_buckets(
    bc: BallControl,
    color_to_x: dict[str, int],
    color_to_event: dict[str, asyncio.Event],
):
    """sort into buckets with claw 0"""

    claw_index = 0
    nof_balls = 6  # to sort into buckets
    right_src_x = 0
    max_y = bc.get_state().max_y
    min_y = max_y + 1 - nof_balls
    for y in range(min_y, max_y + 1):
        color = next(
            ball.color
            for ball in bc.get_state().balls
            if ball.pos == StatePosition(x=right_src_x, y=y)
        )
        
        await go_to_pos(bc=bc, dest=get_column_top_occupied_pos(bc=bc, x=right_src_x), open_claw=True, claw_index=claw_index)
        await bc.close_claw(claw_index=claw_index)
        ev = color_to_event.get(color)
        if ev:
            await ev.wait()
        dest_x = color_to_x[color]
        await go_to_pos(bc=bc, dest=get_column_top_vacant_pos(bc=bc, x=dest_x), open_claw=False, claw_index=claw_index)
        await bc.open_claw(claw_index=claw_index)



In [ ]:
async def challenge12_solution():
    bc = get_control_sim(delay_multiplier=0.3)
    await bc.set_scenario(Ch12Scenario(seed=None))
    
    color_to_x: dict[str, int] = {}
    color_to_event: dict[str, asyncio.Event] = {}

    # create an event for each color
    for color in [ball.color for ball in bc.get_state().balls if ball.pos.x == 0]:
        if color_to_event.get(color) == None:
            color_to_event[color] = asyncio.Event()

    # sort and decode concurrently
    await asyncio.gather(
        reveal_color_values(
            bc=bc, color_to_x=color_to_x, color_to_event=color_to_event, goal_nof_colors=3
        ),
        sort_into_buckets(bc=bc, color_to_x=color_to_x, color_to_event=color_to_event),
    )

    assert bc.get_state().goal_accomplished

In [ ]:
await challenge12_solution()